In [1]:
#%matplotlib notebook
%pylab
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
from mpl_toolkits.mplot3d import axes3d
from matplotlib import cm
import matplotlib as mpl
import matplotlib.animation as animation

import copy

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [2]:
rmin, rmax, Npoints_r = 0.1, 1., 10
thetamin, thetamax, Npoints_theta = 0.1, 0.9*np.pi, 20
phimin, phimax, Npoints_phi = 0., 2.*np.pi, 20

Tmin, Tmax, Npoints_t = 0.,2.0, 200

r = np.linspace(rmin, rmax, Npoints_r)
theta = np.linspace(thetamin, thetamax, Npoints_theta)
phi = np.linspace(phimin, phimax, Npoints_phi)

t = np.linspace(Tmin, Tmax, Npoints_t)

dr = (rmax-rmin)/(Npoints_r-1)
dtheta = (thetamax-thetamin)/(Npoints_theta-1)
dphi = (phimax-phimin)/(Npoints_phi-1)
deltat = (Tmax-Tmin)/(Npoints_t-1)


# Velocidad de la onda

v = 1.0

nu_ = v*deltat/dr
lambda_ = dr/dtheta
mu_ = dr/dphi

print(dr,dphi,dtheta, deltat) 
print(nu_, lambda_, mu_)

0.1 0.3306939635357677 0.1435491256963586 0.010050251256281407
0.10050251256281406 0.6966256291349652 0.30239439187460115


In [3]:
# Definimos condicion inicial para la onda

def ui(r,theta,phi):
   # return 0.3/r
   # return 0.5*np.sin(2*np.pi*r)/r 
    return 0.

u = np.zeros((Npoints_t,Npoints_r,Npoints_theta,Npoints_phi))

for i in range(len(r)):
    for j in range(len(theta)):
        for k in range(len(phi)):
            u[0,i,j,k] = ui(r[i],theta[j],phi[k])

In [4]:
# Fuente de onda

def Source(r,theta,phi):
    return -0.003*np.cos(2*np.pi*r) 

In [5]:
# Zona de evolucion

#i = r , j = theta, k = phi

for l in range(1,len(t)):
    
    
    
    if l==1:
        u[l,:,:,:] = u[l-1,:,:,:] # Si la velocidad de la onda = 0
    else:
        
        u[l,-1,:,:] = 0.3*np.sin(t[l])
        
    
        for i in range(len(r)-1):
            for j in range(len(theta)-1):
                for k in range(len(phi)):
                
                    # Condicion de peridicidad
                    if k == len(phi)-1:
                        p = 1
                    else:
                        p = k+1
                    
                    a = 1.
                    b = dr/r[i]
                    c = (lambda_/r[i])**2
                    d = (dr**2)/(r[i]**2*np.tan(theta[j])*dtheta)
                    e = ( (mu_)/(r[i]*np.sin(theta[j])) )**2
                    
                    #print(a,b,c,d,e)
                    
                    u[l,i,j,k] = (nu_**2)*(  a*(u[l-1,i+1,j,k] - 2*u[l-1,i,j,k] + u[l-1,i-1,j,k]) \
                                           + b*(u[l-1,i+1,j,k] +   u[l-1,i-1,j,k]) \
                                          # + c*(u[l-1,i,j+1,k] - 2*u[l-1,i,j,k] + u[l-1,i,j-1,k]) \
                                          # + d*(u[l-1,i,j+1,k] + u[l-1,i,j-1,k]) \
                                          # + e*(u[l-1,i,j,p] - 2*u[l-1,i,j,k] + u[l-1,i,j,k-1]) \
                                           ) \
                    + 2.*u[l-1,i,j,k] - u[l-2,i,j,k] \
                    #- Source(r[i],theta[j],phi[k]) # Fuente

In [6]:
# Zona de solucion

Nframes = len(t)
fig = plt.figure()
ax = fig.add_subplot(1,1,1, projection = '3d',elev = 30, azim = 50)

lim = 5

def init():
    
    ax.set_xlim3d(-lim, lim)
    ax.set_ylim3d(-lim, lim)
    ax.set_zlim3d(-lim, lim)

    ax.set_xlabel('$x$', fontsize=10)
    ax.set_ylabel('$y$', fontsize=10)
    ax.set_zlabel('$z$', fontsize=10)  
    
def animate(i):
    
    plot = ax.clear()
       
    init()
 
    R, Theta, Phi = np.meshgrid(r,theta,phi, indexing='ij') + u[i,:,:,:]
    
    X,Y,Z = R*np.sin(Theta)*np.cos(Phi), R*np.sin(Theta)*np.sin(Phi), R*cos(Theta) 
    
    plot = ax.scatter3D(X,Y,Z, marker='.', color='b', alpha=0.5)
    fake2Dline = mpl.lines.Line2D([0],[0], linestyle='none', c='b', marker='o')
    plot = ax.legend([fake2Dline], [r'$u(%.3f,r,\theta,\phi)$' %(t[i])], numpoints=1)
    return plot

Animation = animation.FuncAnimation(fig, animate, frames=Nframes, init_func=init)

In [7]:
Writer = animation.writers['ffmpeg']
writer = Writer(fps=10, metadata=dict(artist='Me'), bitrate=1800 )
#Animation.save( 'Onda3D_Esfericas.mp4', writer=writer )